# Bayesian Time Series Analysis of Spotify Listening Time per Day

## Set up 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

# Load the prepared data
spot = pd.read_csv('spot_day_data.csv')

spot = spot[['date', 'minutes_listened']]

## Model Specifications

Proceeding with a Bayesian Structural Time Series model. This model consists of four components: trend, seasonality, regression, and autoregression.

1. **Local level model**: Determines the central tendency of the series. It has one parameter: \( u_t \).
2. **Local linear trend**: Determines the trend of the series. It has two parameters: \( u_t \) (level) and \( v_t \) (slope, which is the rate of change of the level).
3. **Seasonal effect**: Determines the magnitude and pattern of the series. It has one parameter: \( s_t \).

The model is defined by the following equation:

where:
- \( y_t \) is the observed value at time \( t \),  
- \( u_t \) is the local level at time \( t \), 
- \( v_t \) is the local linear trend at time \( t \),  

## Prior Distrib and Prior Pred

$$
u_t \sim \mathcal{N}(0, 10),    
v_t \sim \mathcal{N}(0.5, 1),   
s_t \sim \mathcal{N}(0, 1),     
$$  